<a href="https://colab.research.google.com/github/advapplab/sentiment_analysis_election_2022/blob/main/split_ICI_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/df_sample.xlsx

--2022-12-01 03:46:57--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/df_sample.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111029 (108K) [application/octet-stream]
Saving to: ‘df_sample.xlsx’

df_sample.xlsx      100%[===================>] 108.43K  --.-KB/s    in 0.01s   

2022-12-01 03:46:57 (7.32 MB/s) - ‘df_sample.xlsx’ saved [111029/111029]



In [2]:
import pandas as pd

In [6]:
raw_df = pd.read_excel('/content/df_sample.xlsx')

In [8]:
#insert an extra numbered column. This is the only way to track what sentences belong to the same post. 
import numpy as np
raw_df['Post_number'] = np.arange(len(raw_df))

In [12]:
#split text on '。'
#step 1: split the raw_df into a df that preserves the original posts' labels
#step 2: create split raw_df again into another df that preserves the original posts' Post_number (for our own reference)
#step 3: insert sentiment labels from first df into the second df so we have a split df that shows 
#        both the posts original Post_number and labels
split_for_label_df = pd.DataFrame(raw_df.text.str.split('。').tolist(), index=raw_df.sentiment).stack()
split_for_label_df

sentiment   
neutral    0    【谈北市选战三脚督柯文哲点出“2情况”：蓝绿都投黄珊珊】九合一选举年底登场，台北市长选战“三...
           1                                                     
fear       0    【黄珊珊民调反超陈时中、威胁蒋万安？ 郭正亮：关键在这群人】对于有台北市长选举民调显示，蒋万...
           1                       民众党台北市议员参选人吴怡萱认为民进党有点误判情势，自乱阵脚
           2          前民进党立委郭正亮则认为未来关键在于40岁以下的选民如何投票，所以蒋万安勤奋跑亲子行程
                                      ...                        
surprise   0    【周玉蔻怎能霸占黄金时段？郭正亮爆：被“有分量”金主标下】资深媒体人周玉蔻日前攻击国民党台北...
           1    对此，前立委郭正亮在《朱学恒的阿宅万事通事务所》节目中指出，周玉蔻节目被移回152台民视台湾...
           2                                                     
           0                                   额？？？这是蒋万安的竞选团队？？？ 
angry      0    分享网易新闻《周玉蔻再出手！明日将状告柯文哲蒋万安诽谤，柯文哲：喜欢告谁就赶快去告》 <a ...
Length: 735, dtype: object

In [13]:
split_for_label_df = split_for_label_df.reset_index([0, 'sentiment',])

In [14]:
#set column back to sentiment and text
split_for_label_df.columns = ['sentiment', 'text']

In [15]:
#delete empty rows
split_for_label_df = split_for_label_df[split_for_label_df['text'].str.len()>=1]
split_for_label_df = split_for_label_df.reset_index()

In [16]:
split_for_label_df

,index,sentiment,text
0,0,neutral,【谈北市选战三脚督柯文哲点出“2情况”：蓝绿都投黄珊珊】九合一选举年底登场，台北市长选战“三...
1,2,fear,【黄珊珊民调反超陈时中、威胁蒋万安？ 郭正亮：关键在这群人】对于有台北市长选举民调显示，蒋万...
2,3,fear,民众党台北市议员参选人吴怡萱认为民进党有点误判情势，自乱阵脚
3,4,fear,前民进党立委郭正亮则认为未来关键在于40岁以下的选民如何投票，所以蒋万安勤奋跑亲子行程
4,5,fear,
...,...,...,...
677,728,neural,【外传绿下周征召陈时中选台北市 蒋万安呛：任何个人决定都不该影响防疫】国民党台北市长参选人蒋...
678,730,surprise,【周玉蔻怎能霸占黄金时段？郭正亮爆：被“有分量”金主标下】资深媒体人周玉蔻日前攻击国民党台北...
679,731,surprise,对此，前立委郭正亮在《朱学恒的阿宅万事通事务所》节目中指出，周玉蔻节目被移回152台民视台湾...
680,733,surprise,额？？？这是蒋万安的竞选团队？？？


In [17]:
#step 2: create split raw_df again into another df that preserves the original posts' Post_number (for our own reference)
challenge_split_df = pd.DataFrame(raw_df.text.str.split('。').tolist(), index=raw_df.Post_number).stack()
challenge_split_df

Post_number   
0            0    【谈北市选战三脚督柯文哲点出“2情况”：蓝绿都投黄珊珊】九合一选举年底登场，台北市长选战“三...
             1                                                     
1            0    【黄珊珊民调反超陈时中、威胁蒋万安？ 郭正亮：关键在这群人】对于有台北市长选举民调显示，蒋万...
             1                       民众党台北市议员参选人吴怡萱认为民进党有点误判情势，自乱阵脚
             2          前民进党立委郭正亮则认为未来关键在于40岁以下的选民如何投票，所以蒋万安勤奋跑亲子行程
                                        ...                        
197          0    【周玉蔻怎能霸占黄金时段？郭正亮爆：被“有分量”金主标下】资深媒体人周玉蔻日前攻击国民党台北...
             1    对此，前立委郭正亮在《朱学恒的阿宅万事通事务所》节目中指出，周玉蔻节目被移回152台民视台湾...
             2                                                     
198          0                                   额？？？这是蒋万安的竞选团队？？？ 
199          0    分享网易新闻《周玉蔻再出手！明日将状告柯文哲蒋万安诽谤，柯文哲：喜欢告谁就赶快去告》 <a ...
Length: 735, dtype: object

In [18]:
challenge_split_df = challenge_split_df.reset_index([0, 'Post_number',])

In [19]:
#set column back to Post_number and text
challenge_split_df.columns = ['Post_number', 'text']

In [20]:
#delete empty rows
challenge_split_df = challenge_split_df[challenge_split_df['text'].str.len()>=1]
challenge_split_df = challenge_split_df.reset_index()

In [21]:
challenge_split_df

,index,Post_number,text
0,0,0,【谈北市选战三脚督柯文哲点出“2情况”：蓝绿都投黄珊珊】九合一选举年底登场，台北市长选战“三...
1,2,1,【黄珊珊民调反超陈时中、威胁蒋万安？ 郭正亮：关键在这群人】对于有台北市长选举民调显示，蒋万...
2,3,1,民众党台北市议员参选人吴怡萱认为民进党有点误判情势，自乱阵脚
3,4,1,前民进党立委郭正亮则认为未来关键在于40岁以下的选民如何投票，所以蒋万安勤奋跑亲子行程
4,5,1,
...,...,...,...
677,728,196,【外传绿下周征召陈时中选台北市 蒋万安呛：任何个人决定都不该影响防疫】国民党台北市长参选人蒋...
678,730,197,【周玉蔻怎能霸占黄金时段？郭正亮爆：被“有分量”金主标下】资深媒体人周玉蔻日前攻击国民党台北...
679,731,197,对此，前立委郭正亮在《朱学恒的阿宅万事通事务所》节目中指出，周玉蔻节目被移回152台民视台湾...
680,733,198,额？？？这是蒋万安的竞选团队？？？


In [22]:
#step 3: take sentiment labels from the split_for_label_df and insert them into challenge_split_df for final dataframe that we can use for testing
extracted_col2 = split_for_label_df["sentiment"]
challenge_split_df.insert(1, "sentiment", extracted_col2)

In [23]:
#mission accomplished, I verified the output with the original excel sheet and it all checks out.
challenge_split_df

,index,sentiment,Post_number,text
0,0,neutral,0,【谈北市选战三脚督柯文哲点出“2情况”：蓝绿都投黄珊珊】九合一选举年底登场，台北市长选战“三...
1,2,fear,1,【黄珊珊民调反超陈时中、威胁蒋万安？ 郭正亮：关键在这群人】对于有台北市长选举民调显示，蒋万...
2,3,fear,1,民众党台北市议员参选人吴怡萱认为民进党有点误判情势，自乱阵脚
3,4,fear,1,前民进党立委郭正亮则认为未来关键在于40岁以下的选民如何投票，所以蒋万安勤奋跑亲子行程
4,5,fear,1,
...,...,...,...,...
677,728,neural,196,【外传绿下周征召陈时中选台北市 蒋万安呛：任何个人决定都不该影响防疫】国民党台北市长参选人蒋...
678,730,surprise,197,【周玉蔻怎能霸占黄金时段？郭正亮爆：被“有分量”金主标下】资深媒体人周玉蔻日前攻击国民党台北...
679,731,surprise,197,对此，前立委郭正亮在《朱学恒的阿宅万事通事务所》节目中指出，周玉蔻节目被移回152台民视台湾...
680,733,surprise,198,额？？？这是蒋万安的竞选团队？？？
